### 1. 열 기준 병합: 고객데이터랑 구매데이터 통합해서 연령대 별 평균 구매액 분석하기

In [ ]:
# 사전준비: 필요 라이브러리들 import 해오기
from pandas import DataFrame
from pandas import merge
from hossam import load_data
from matplotlib import pyplot as plt
from sklearn.impute import SimpleImputer
import numpy as np

In [ ]:
# 사전준비: 주어진 데이터로 고객정보 데이터프레임 생성:

df_customer = DataFrame(
    {
        'customer_id': [101, 102, 103, 104, 105, 106],
        'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank'],
        'age': [25, 32, 45, 38, 29, 41]
    }
)
df_customer

,customer_id,name,age
0,101,Alice,25
1,102,Bob,32
2,103,Charlie,45
3,104,David,38
4,105,Eve,29
5,106,Frank,41


In [ ]:
# 사전준비: 구매내역 데이터프레임은 불러오기
order_history = load_data('order_history')
order_history

[data] https://data.hossam.kr/data/lab04/order_history.xlsx
[desc] 신입 데이터 분석가를 위한 실전 미션!

field        description
-----------  -------------
purchase_id  일련번호
customer_id  고객번호
item         상품명
price        금액



,customer_id,item,price
purchase_id,,,
1,101,Laptop,1200.0
2,102,Mouse,25.0
3,103,Keyboard,NaN
4,101,Monitor,300.0
5,104,Webcam,5000.0
6,102,USB Hub,30.0
7,105,Laptop,1100.0
8,103,Mouse,NaN
9,101,External HDD,80.0


In [ ]:
# 요구사항 1: 구매데이터의 price 컬럼에 있는 결측치를 상품 평균가격으로 대체하기
df_pNaToMean = order_history.fillna(
    value = order_history['price'].mean()
    )
df_pNaToMean

,customer_id,item,price
purchase_id,,,
1,101,Laptop,1200.000
2,102,Mouse,25.000
3,103,Keyboard,985.625
4,101,Monitor,300.000
5,104,Webcam,5000.000
6,102,USB Hub,30.000
7,105,Laptop,1100.000
8,103,Mouse,985.625
9,101,External HDD,80.000


In [ ]:
# 요구사항 2: price가 1000 초과하는 데이터(이상치로 간주)는 상품의 중앙값으로 대체하기
df2 = df_pNaToMean.copy()
df2.loc[df2['price'] > 1000, 'price'] = np.nan

df3 = df2.fillna(
    value = df2['price'].median()
)
df3

,customer_id,item,price
purchase_id,,,
1,101,Laptop,150.000
2,102,Mouse,25.000
3,103,Keyboard,985.625
4,101,Monitor,300.000
5,104,Webcam,150.000
6,102,USB Hub,30.000
7,105,Laptop,150.000
8,103,Mouse,985.625
9,101,External HDD,80.000


In [ ]:
# 요구사항 3: 만든 고객정보랑 정제한 구매내역 데이터프레임 customer_id 기준으로 병합하기
df4 = merge(df_customer, df3, on = 'customer_id')
df4

,customer_id,name,age,item,price
0,101,Alice,25,Laptop,150.000
1,101,Alice,25,Monitor,300.000
2,101,Alice,25,External HDD,80.000
3,102,Bob,32,Mouse,25.000
4,102,Bob,32,USB Hub,30.000
5,103,Charlie,45,Keyboard,985.625
6,103,Charlie,45,Mouse,985.625
7,104,David,38,Webcam,150.000
8,105,Eve,29,Laptop,150.000
9,106,Frank,41,Docking Station,150.000


In [11]:
# 요구사항 4: 병합 데이터프레임의 기술통계랑 도출하기
df5 = df4.astype(
    {'customer_id': 'str'}
)
df5.describe()

,age,price
count,10.00000,10.000000
mean,33.70000,300.625000
std,8.04225,369.246135
min,25.00000,25.000000
25%,26.00000,97.500000
50%,32.00000,150.000000
75%,40.25000,262.500000
max,45.00000,985.625000


### 2. 행 기준 병합: 흩어진 판매데이터 통합하기

In [9]:
# 사전준비: 필요 라이브러리들 import 해오기
from pandas import DataFrame
from pandas import concat
from hossam import load_data
from matplotlib import pyplot as plt
from sklearn.impute import SimpleImputer
import numpy as np

In [2]:
# 사전준비: shop_강남, shop_종로 데이터프레임은 불러오기
shop_강남 = load_data('shop_강남')
shop_강남

[data] https://data.hossam.kr/data/lab04/shop_강남.xlsx
[desc] 흩어진 판매 데이터 통합하기(인덱스/메타데이터 없음)
[!] Cannot read metadata


,OrderID,Product,Quantity,Price
0,A001,Keyboard,10,15000.0
1,A002,Mouse,20,NaN
2,A003,Monitor,5,75000.0
3,A004,Keyboard,12,15000.0


In [3]:
shop_종로 = load_data('shop_종로')
shop_종로

[data] https://data.hossam.kr/data/lab04/shop_종로.xlsx
[desc] 흩어진 판매 데이터 통합하기(인덱스/메타데이터 없음)
[!] Cannot read metadata


,OrderID,Product,Quantity,Price
0,B001,Mouse,15,8000
1,B002,Keyboard,999,14500
2,B003,Webcam,10,30000
3,B004,Mouse,18,8000


In [ ]:
# 1. 강남점 데이터의 mouse 가격 8000으로 바꾸기:
shop_강남['Price'][1] = 8000
shop_강남
#강남점 = shop_강남.fillna(8000)
# 또는
#강남점['Price'] = shop_강남['Price'].fillna(8000)

C:\Users\itwill\AppData\Local\Temp\ipykernel_16776\3685326466.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  shop_강남['Price'][1] = 8000
C:\Users\itwill\AppData\Local\Temp\ipykernel_16776\3685326466.py:1: SettingWithCopyWarning: 
A value

,OrderID,Product,Quantity,Price
0,A001,Keyboard,10,15000.0
1,A002,Mouse,20,8000.0
2,A003,Monitor,5,75000.0
3,A004,Keyboard,12,15000.0


In [ ]:
# 2. 종로점 데이터의 keyboard quantity 9로 바꾸기:
shop_종로['Quantity'][1] = 9
shop_종로

#----------방법1------------
#종로점cp = shop_종로.copy()
#idx = 종로점cp[종로점cp['Product'] == 'keyboard'].index[0]
#종로점cp

#----------방법2------------
#종로점cp = shop_종로.copy()
#종로점cp['Quantity'] = 종로점cp['Quantity'].replace(999, 9)
# -> 근데 이건 만약 999값이 여러개 있으면 일괄 변경 되니까 케바케

C:\Users\itwill\AppData\Local\Temp\ipykernel_16776\1332668141.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  shop_종로['Quantity'][1] = 9
C:\Users\itwill\AppData\Local\Temp\ipykernel_16776\1332668141.py:1: SettingWithCopyWarning: 
A value

,OrderID,Product,Quantity,Price
0,B001,Mouse,15,8000
1,B002,Keyboard,9,14500
2,B003,Webcam,10,30000
3,B004,Mouse,18,8000


In [ ]:
# 3. 강남점이랑 종로점 데이터프레임 하나로 합치기 (인덱스 재구성)
df1 = concat([shop_강남, shop_종로])
df1.reset_index(inplace = True, drop = True)
df1

#------------------다른방법----------------------
#df_total = concat([shop_강남, shop_종로], ignore_index = True)
#df_total

,OrderID,Product,Quantity,Price
0,A001,Keyboard,10,15000.0
1,A002,Mouse,20,8000.0
2,A003,Monitor,5,75000.0
3,A004,Keyboard,12,15000.0
4,B001,Mouse,15,8000.0
5,B002,Keyboard,9,14500.0
6,B003,Webcam,10,30000.0
7,B004,Mouse,18,8000.0


### 3. 정렬, 필터링: 마케팅 신입사원의 첫 번째 미션

In [ ]:
# 사전준비: 라이브러리 import
from hossam import load_data

In [ ]:
# 사전준비: 데이터 불러오고 정보보기
vips = load_data('vip_customer')
vips.info()

[data] https://data.hossam.kr/data/lab04/vip_customer.xlsx
[desc] VIP 고객 목록 데이터

field             description
----------------  ----------------
CustomerID        고객번호
Name              고객명
Age               나이
City              거주지역
JoinDate          가입일
TotalSpending     총 지출액
LastPurchaseDate  마지막 구매 날짜

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 1001 to 1015
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Name              15 non-null     object        
 1   Age               15 non-null     int64         
 2   City              15 non-null     object        
 3   JoinDate          15 non-null     datetime64[ns]
 4   TotalSpending     15 non-null     int64         
 5   LastPurchaseDate  15 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 840.0+ bytes


In [ ]:
# 1. 제시한 열 3개만 필터링하기
vips1 = vips.filter(['Name', 'City', 'TotalSpending'])
vips1

,Name,City,TotalSpending
CustomerID,,,
1001,김철수,서울,150000
1002,이영희,부산,80000
1003,박민준,서울,250000
1004,최지우,인천,50000
1005,정다솜,서울,320000
1006,윤현우,부산,120000
1007,강예진,서울,85000
1008,오승민,광주,180000
1009,한지민,서울,450000


In [ ]:
# 2. 총 구매액 100,000원 이상인 고객만 필터하기
vips2 = vips1.query(
    'TotalSpending >= 100000 and City == "서울"')
vips2

,Name,City,TotalSpending
CustomerID,,,
1001,김철수,서울,150000
1003,박민준,서울,250000
1005,정다솜,서울,320000
1009,한지민,서울,450000
1011,임도윤,서울,210000
1013,황재민,서울,190000
1015,고은별,서울,130000


In [24]:
# 3. 총 구매액 높은 순서대로 정렬 후 상위 5명만 추출하기
fiveVips = vips2.sort_values(
    'TotalSpending', ascending = False
    ).head() # 메서드체인: .메서드1().메서드2()...
fiveVips

,Name,City,TotalSpending
CustomerID,,,
1009,한지민,서울,450000
1005,정다솜,서울,320000
1003,박민준,서울,250000
1011,임도윤,서울,210000
1013,황재민,서울,190000


### 4. 그룹집계: 온라인 쇼핑몰의 다음 분기 프로모션 전략 수립

In [1]:
from hossam import load_data

In [2]:
df_order = load_data('shop_order_list')
df_order.info()

[data] https://data.hossam.kr/data/lab04/shop_order_list.xlsx
[desc] 온라인 쇼핑몰의 최근 고객 주문 데이터
[!] Cannot read metadata
<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 1001 to 1020
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   UserID     20 non-null     object        
 1   Product    20 non-null     object        
 2   Price      18 non-null     float64       
 3   Category   20 non-null     object        
 4   Region     20 non-null     object        
 5   OrderDate  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 1.1+ KB


In [ ]:
df1 = df_order.copy()
df_pNaToMean = df1.fillna(
    value = df1['Price'].mean()
    )
df_pNaToMean

,UserID,Product,Price,Category,Region,OrderDate
OrderID,,,,,,
1001,user001,Laptop,1200.000000,Electronics,Seoul,2023-01-15
1002,user002,Mouse,25.000000,Electronics,Busan,2023-01-16
1003,user003,Keyboard,70.000000,Electronics,Seoul,2023-01-17
1004,user001,Book,15.000000,Books,Incheon,2023-01-18
1005,user004,T-shirt,30.000000,Apparel,Gwangju,2023-01-19
1006,user002,Book,20.000000,Books,Seoul,2023-01-20
1007,user005,Jeans,80.000000,Apparel,Busan,2023-02-21
1008,user001,Monitor,300.000000,Electronics,Seoul,2023-02-22
1009,user006,Book,18.000000,Books,Daegu,2023-02-23


In [5]:
df2 = df_pNaToMean.copy()
df2.filter(['UserID', 'Category']).groupby('Category').sum()

,UserID
Category,
Apparel,user004user005user007user010user003
Books,user001user002user006user005user007
Electronics,user001user002user003user001user003user002
Food,user008user009
Furniture,user001user004
